In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import math
plt.rcParams["animation.html"] = "jshtml"
%pylab inline
b = pd.read_csv("stolzenburg_marshall.csv")

In [ ]:
#Define Constants
n = 360
tmax = 4600
dt = 10 #seconds
e0 = 8.854187817e-12
asc_rate = .00444 

#Set Altitude
z = linspace(0,16,n+1) #"n+1" creates a layer with width 1/n
threshold_upper = 2.84e5*exp(-z[1:]/8) #2.84e5 is the base Electric Field required for Relativistic Electron Avalanche
threshold_lower = 1e5*exp(-z[1:]/8) #This threshold is arbitrary, however, it is important that the threshold is not too low
    
#Define Functions
def f(z): #Function for Current      get rid of complicated current, get simple current (not modeling real storm)
    return -.125e-9*exp(-0.5*(z-4)**2) + 1.5e-9*exp(-.75*(z-7.5)**2) + -.5e-9*exp(-0.75*(z-13.75)**2)

def calcField(sigma):
    return A.dot(sigma) #Dot product between "A" and Charge Density

def discharge(sigma,E):
    if any(abs(E) > threshold_upper):
        dE = (abs(E) - threshold_upper)/threshold_upper #A ratio of Electric Field to Threshold
        dE_above = dE[dE>0] #An array that includes the dE values where the Electric Field is greater than the threshold
        Ebig = where(abs(E) > threshold_upper)[0]
        zz = z[Ebig]     ## z where E is above threshold
            
        for i in range(len(zz)):
            discharge_prob = rand()
            if discharge_prob < 1 - exp(dt*-dE_above[i]**2): 
                
                lightning = (-1e-7/dt)*exp(-.25*(z-zz[i])**4)  ## current centered where lightning is occuring
                sigma[:] = sigma + sign(E[Ebig[i]])*diff(lightning)*dt
                print(round(zz[i],3), t)
                E[:] = calcField(sigma)
                    
                if any(abs(E) > threshold_lower):
                    kz = Ebig[i] #where(zz[i] == z)
                    
                    while abs(E[kz]) > threshold_lower[kz]:
                        kk = 1
                        if abs(E[kz]) > threshold_lower[kz]:
                                
                            lightning2 = (-1e-8/dt)*exp(-1*(z-z[kz])**2)
                            sigma[:] = sigma + sign(E[kz])*diff(lightning2)*dt
                            print("    ", round(z[kz],2))
                            E[:] = calcField(sigma)
                            
                        while abs(E[kz+kk]) > threshold_lower[kz+kk] or abs(E[kz-kk]) > threshold_lower[kz-kk]:
                                
                            if abs(E[kz+kk]) > threshold_lower[kz+kk]:
                                
                                lightning2_up = (-1e-8/dt)*exp(-1*(z-z[kz+kk])**2)
                                sigma[:] = sigma + sign(E[kz+kk])*diff(lightning2_up)*dt
                                print("    ", round(z[kz+kk],2))
                                E[:] = calcField(sigma)
                                
                            if abs(E[kz-kk]) > threshold_lower[kz-kk]:
                                
                                lightning2_down = (-1e-8/dt)*exp(-1*(z-z[kz-kk])**2)
                                sigma[:] = sigma + sign(E[kz-kk])*diff(lightning2_down)*dt
                                print("    ", round(z[kz-kk],2))
                                E[:] = calcField(sigma)
                            
                            kk = kk + 1 
                    print()

I = f(z)

#Matrix used to calculate the Electric Field from the Charge Density
a = -tri(n, k=n) + tri(n) + tri(n,k=-1)
A = a*(1/(2*e0)) #Electric field of a sheet of charge (probably not fringe field)

In [ ]:
#Reset the time for every run through of the simulation
t = 0 #seconds

#Define Arrays and Lists
E = np.zeros(n) #Creates an array of "n" values... stays consistent with layer width
sigma = np.zeros(n) #Same reason as above
E_field = [] #Library for Electric Field profiles at each time step
sigma_array = [] #Library for Charge Density profiles at each time step

while t < tmax:
    sigma = sigma - diff(I)*dt

    #Electric Field
    E = calcField(sigma)
    E_field.append(copy(E))
        
    #For Lightning Discharges
    discharge(sigma,E)
                
    sigma_array.append(copy(sigma))
    t = t + dt

In [ ]:
#For data collection
t = 0
payloads = 1 #Number of data collections
delay = 1000
jj = 0
balloon_z = empty(payloads)

#Set Payload Release Times
release_time = [[] for i in range(payloads)]
for j in range(payloads):
    release_time[j].append(30*j + delay) #Release payloads every 30 seconds, starting after a specified delay

#Define Starting Altitude for Payloads
for j in range(payloads):
    balloon_z[j] = 0

stored_data = [[] for i in range(payloads)]
stored_height = [[] for i in range(payloads)]
stored_time = [[] for i in range(payloads)]

while t < tmax:
    balloon_data = empty((tmax-delay)//dt)
    for j in range(payloads):
        if t > release_time[j][0]:
            balloon_data[j] = np.interp(balloon_z[j],z[1:],E_field[jj])
            balloon_z[j] = balloon_z[j] + asc_rate*dt
            stored_data[j].append(balloon_data[j])
            stored_height[j].append(balloon_z[j])
            stored_time[j].append(t)
    jj = jj + 1  
    t = t + dt

In [ ]:
for j in range(payloads):
    plot(stored_data[j],stored_height[j], label="Payload " + str([j+1]))
plot(b['x'],b['y'],label="Literature Data")
plt.xlabel("E (V/m)")
plt.ylabel("Altitude (km)")
legend(loc="upper left")
figsize(8,8)

In [ ]:
for j in range(payloads):
    plot(stored_data[j],stored_time[j])
plt.xlabel("E (V/m)")
plt.ylabel("Time (s)")
figsize(8,8)

fig = plt.figure()
ax = plt.axes(xlim=(-1e5,1e5), ylim=(0,16))
line, = ax.plot([], [], lw=2)
plot(b['x'],b['y'])
plot(threshold_upper, z[1:])
plot(-threshold_upper, z[1:])
plot(threshold_lower, z[1:])
plot(-threshold_lower, z[1:])
plt.xlabel("E (V/m)")
plt.ylabel("Altitude (km)")
plt.close()

def init():
    line.set_data([], [])
    return line,
def animate(i):
    line.set_data(E_field[i],z[1:])
    return line,

animation.FuncAnimation(fig, animate, init_func=init, frames=360, interval=75, blit=True)

Leaving this out for now so the notebook runs well